In [1]:
from typing import Tuple, cast
from pydbsp.zset import ZSet

Edge = Tuple[int, int]
GraphZSet = ZSet[Edge]

def load_graph(file_path: str) -> GraphZSet:
    out = []

    with open(file_path, mode="r") as file:
        lines = file.readlines()
        out = {
            edge: 1
            for edge in map(
                lambda line: tuple(map(lambda node: int(node), line.split()[0:2])),
                lines,
            )
        }

    return cast(GraphZSet, ZSet(out))

In [2]:
dense_graph_zset = load_graph("data/graph1000.txt")
sparse_graph_zset = load_graph("data/graph10000.txt")

In [3]:
from pydbsp.stream import Stream, StreamHandle
from pydbsp.zset import ZSetAddition
from pydbsp.lazy_zset import LazyZSet, LazyZSetAddition

In [4]:
dense_graph_stream = Stream(ZSetAddition())
dense_graph_stream.send(dense_graph_zset)
dense_graph_stream_handle = StreamHandle(lambda: dense_graph_stream)

In [5]:
sparse_graph_stream = Stream(ZSetAddition())
sparse_graph_stream.send(sparse_graph_zset)
sparse_graph_stream_handle = StreamHandle(lambda: sparse_graph_stream)

In [7]:
from pydbsp.algorithms.graph_reachability import IncrementalGraphReachability

zset_dense_graph_reachability = IncrementalGraphReachability(dense_graph_stream_handle)
zset_sparse_graph_reachability = IncrementalGraphReachability(sparse_graph_stream_handle)
#lazy_zset_dense_graph_reachability = LazyIncrementalGraphReachability(lazy_dense_graph_stream_handle)

In [8]:
from pydbsp.stream import step_until_fixpoint
from pydbsp.stream.functions.linear import stream_elimination

In [9]:
%time step_until_fixpoint(zset_dense_graph_reachability)

CPU times: user 1.41 s, sys: 33.1 ms, total: 1.44 s
Wall time: 1.44 s


In [10]:
len(stream_elimination(zset_dense_graph_reachability.flattened_output.output()).inner)

11532

In [11]:
%time step_until_fixpoint(zset_sparse_graph_reachability)

CPU times: user 1min 15s, sys: 220 ms, total: 1min 16s
Wall time: 1min 16s


In [ ]:
len(stream_elimination(zset_sparse_graph_reachability.output()).inner)